In [ ]:
import numpy as np

from reachy_sdk import ReachySDK

from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode
import time

In [ ]:
reachy = ReachySDK('192.168.1.21')

Check if all the joints seem to be detected.

In [ ]:
for j in reachy.joints.values():
    j.torque_limit = 0

## Checking compliance

In [ ]:
reachy.joints

In [ ]:
reachy.head.look_at(0.5,0.0,0.0,duration=3.0)

Lift both arms, put them in stiff mode and check if *turn_off_smoothly* works.

In [ ]:
reachy.turn_on('reachy')

In [ ]:
reachy.turn_off_smoothly('reachy')

## Moving the arms

In [ ]:
from scipy.spatial.transform import Rotation as R
import numpy as np

R.from_matrix([
    [0, 0, -1],
    [0, 1, 0],
    [1, 0, 0],
]).as_euler('xyz', degrees=True)


In [ ]:
from scipy.spatial.transform import Rotation as R
import numpy as np
def create_target_xyz_roll_yaw(x, y, z, roll, yaw) :
    y_rot = np.around(R.from_euler('y', np.deg2rad(-90)).as_matrix(), 3)
    #print(y_rot)
    x_rot = np.around(R.from_euler('z', np.deg2rad(roll)).as_matrix(), 3)
    z_rot = np.around(R.from_euler('x', np.deg2rad(yaw)).as_matrix(), 3)
    #print(x_rot)
    total_rot = y_rot@x_rot@z_rot
    #print(total_rot)
    total_rot=np.r_[ total_rot, [np.array([1, 1, 1])] ]
    #print(total_rot)
    total_rot=np.c_[ total_rot, np.array([x, y, z, 1]) ] 
    #print(total_rot)
    return total_rot

In [ ]:
print(create_target_xyz_roll_yaw(0.3, 0.15, -0.4, 0, 0))

For the left arm:

In [ ]:
A = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.15],  
  [1, 0, 0, -0.40],
  [0, 0, 0, 1],  
])

B = np.array([
  [0, 0, -1, 0.3],
  [0, 1, 0, 0.15],  
  [1, 0, 0, -0.445],
  [0, 0, 0, 1],  
])

In [ ]:
A=create_target_xyz_roll_yaw(0.3, 0.125, -0.4, 0, -20)
B=create_target_xyz_roll_yaw(0.3, 0.125, -0.435, 0, -20)

In [ ]:
targets = []
targets.append(create_target_xyz_roll_yaw(0.3, 0.20, -0.4, -4, -20))
targets.append(create_target_xyz_roll_yaw(0.3, 0.20, -0.445, -4, -20))
targets.append(create_target_xyz_roll_yaw(0.3, 0.17, -0.4, -4, -20))
targets.append(create_target_xyz_roll_yaw(0.3, 0.17, -0.445, -4, -20))
targets.append(create_target_xyz_roll_yaw(0.3, 0.14, -0.4, -0, -20))
targets.append(create_target_xyz_roll_yaw(0.3, 0.14, -0.445, -0, -20))

In [ ]:
# put the joints in stiff mode
try:
    reachy.turn_on('l_arm')

    # use the goto function
    for i in range(6) :
        for t in targets :
            goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(),reachy.l_arm.inverse_kinematics(t) )}, duration=0.5)
            time.sleep(0.3)
    
    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), reachy.l_arm.inverse_kinematics(targets[0]))}, duration=1.0)
    time.sleep(0.2)
    # put the joints back to compliant mode
    # use turn_off_smoothly to prevent the arm from falling hard
    #reachy.turn_off_smoothly('l_arm')
except AttributeError:
    print('Reachy has no left arm!')

In [ ]:
# put the joints in stiff mode
try:
    reachy.turn_on('l_arm')

    goto({joint: pos for joint,pos in zip(reachy.l_arm.joints.values(), reachy.l_arm.inverse_kinematics(targets[0]))}, duration=1.0)
    time.sleep(0.2)
    # put the joints back to compliant mode
    # use turn_off_smoothly to prevent the arm from falling hard
    #reachy.turn_off_smoothly('l_arm')
except AttributeError:
    print('Reachy has no left arm!')

In [ ]:
reachy.turn_off_smoothly('reachy')

In [ ]:
A = np.array([
    [0, 0, -1, 0.3],
    [0, 1, 0, 0.15],
    [1, 0, 0, -0.40],
    [0, 0, 0, 1],
])

B = np.array([
    [0, 0, -1, 0.3],
    [0, 1, 0, 0.15],
    [1, 0, 0, -0.445],
    [0, 0, 0, 1],
])
# put the joints in stiff mode
try:
    reachy.turn_on('l_arm')

    goto({joint: pos for joint, pos in zip(reachy.l_arm.joints.values(),
         reachy.l_arm.inverse_kinematics(A))}, duration=2)

except AttributeError:
    print('Reachy has no left arm!')